In [1]:
#history
# tushare读数据 
import tushare as ts
ts.set_token('0742c95bee169abeccdf80bbdce1b2e0cf2ac4d83cbd932b608300fd')
pro = ts.pro_api()

def get_price(tickers, start, end):
    stocks = pro.daily(ts_code=tickers[0], start_date=start, end_date=end, fields=['trade_date'])
    for ticker in tickers:
        stock = pro.daily(ts_code=ticker, start_date=start, end_date=end, fields=['trade_date', 'close'])
        stock = stock.rename(columns={"close" : ticker})
        stocks = stocks.merge(stock,on=['trade_date'])
    stocks.set_index(['trade_date'], inplace=True)
    stocks = stocks.iloc[::-1]
    stocks = stocks.dropna()
    return stocks

def get_date_cv(tickers, date):
    stocks = pro.daily(ts_code=tickers[0], trade_date=date, fields=['ts_code', 'close', 'vol'])
    for ticker in tickers:
        stock = pro.daily(ts_code=ticker, trade_date=date, fields=['ts_code', 'close', 'vol'])
        stocks = stocks.append(stock)
    return stocks

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import statsmodels.api as sm
# 获取ticker（上证50）
import json

with open('510050_20230214.json') as user_file:
  parsed_json = json.load(user_file)

tickers = []
for i in parsed_json["StockComponent"].keys():
  tickers.append(i+".SH")

#数据准备

start = '20200610'
end = '20230216'
trade_date = end
num_equities = 45
num_components = 3

#coarse selection
coarse = get_date_cv(tickers, trade_date)
# Sort the equities in DollarVolume decendingly
symbols = list(coarse[coarse['close'] > 5].sort_values('vol', ascending=[False]).head(num_equities)['ts_code'])

history = get_price(symbols, start, end)
history

,601668.SH,600905.SH,601857.SH,601899.SH,601669.SH,600031.SH,600048.SH,601919.SH,600030.SH,601166.SH,...,603501.SH,603986.SH,601628.SH,688599.SH_x,688599.SH_y,601995.SH,600309.SH,603288.SH,603260.SH,600809.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
20210610,4.86,3.82,4.91,11.07,3.85,29.32,13.05,25.03,25.66,22.03,...,285.17,141.60,34.64,19.60,19.60,60.25,114.16,136.16,64.10,487.31
20210611,4.92,4.20,5.19,11.00,3.87,28.29,13.07,25.88,24.94,21.56,...,289.47,141.00,33.30,20.50,20.50,61.00,114.80,134.68,65.30,470.25
20210615,4.86,4.62,5.23,10.47,3.80,28.10,12.68,26.10,24.58,20.92,...,301.00,137.87,33.27,20.97,20.97,60.83,114.72,131.45,64.67,486.00
20210616,4.88,4.95,5.31,9.94,3.83,26.92,12.89,24.75,24.65,20.89,...,279.90,135.27,33.70,19.99,19.99,59.18,110.01,133.90,62.33,480.98
20210617,4.83,5.45,5.23,9.87,3.80,27.08,12.89,27.13,24.54,20.80,...,296.35,148.80,33.57,20.46,20.46,60.34,107.81,131.57,66.70,497.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230210,5.43,5.71,5.14,11.19,6.97,16.53,15.78,10.83,20.96,17.19,...,94.00,116.55,34.23,65.88,65.88,39.96,97.99,77.70,97.80,287.01
20230213,5.46,5.70,5.15,11.30,7.05,18.18,15.57,10.77,20.93,16.95,...,94.40,117.05,34.60,64.26,64.26,40.08,97.97,78.87,96.69,296.70
20230214,5.49,5.67,5.17,11.57,7.03,18.34,15.60,10.79,20.92,17.05,...,93.87,115.86,35.23,64.87,64.87,40.21,100.00,78.87,97.58,299.50


In [2]:
#sample
# 数据log平滑处理和中心化处理后做PCA
# Sample data for PCA (smooth it using np.log function)
sample = np.log(history)   #???取对数的效果更好吗?
#sample.mean()对所有symbol的每个交易日求均值
sample -= sample.mean() # Center it column-wise
sample

,601668.SH,600905.SH,601857.SH,601899.SH,601669.SH,600031.SH,600048.SH,601919.SH,600030.SH,601166.SH,...,603501.SH,603986.SH,601628.SH,688599.SH_x,688599.SH_y,601995.SH,600309.SH,603288.SH,603260.SH,600809.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
20210610,-0.060518,-0.486616,-0.061126,0.097398,-0.599973,0.403988,-0.139616,0.486100,0.164256,0.159971,...,0.550199,0.053747,0.146005,-1.116701,-1.116701,0.308735,0.186848,0.399890,-0.575192,0.509520
20210611,-0.048248,-0.391782,-0.005666,0.091054,-0.594791,0.368227,-0.138085,0.519496,0.135796,0.138405,...,0.565166,0.049501,0.106553,-1.071805,-1.071805,0.321106,0.192438,0.388961,-0.556645,0.473884
20210615,-0.060518,-0.296472,0.002011,0.041673,-0.613045,0.361488,-0.168379,0.527961,0.121256,0.108271,...,0.604224,0.027052,0.105652,-1.049137,-1.049137,0.318315,0.191741,0.364686,-0.566339,0.506828
20210616,-0.056411,-0.227479,0.017192,-0.010274,-0.605181,0.318588,-0.151953,0.474851,0.124100,0.106836,...,0.531546,0.008014,0.118494,-1.096998,-1.096998,0.290816,0.149818,0.383153,-0.603194,0.496445
20210617,-0.066710,-0.131251,0.002011,-0.017341,-0.613045,0.324514,-0.151953,0.566665,0.119627,0.102518,...,0.588655,0.103344,0.114629,-1.073758,-1.073758,0.310227,0.129617,0.365599,-0.535432,0.529632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230210,0.050383,-0.084647,-0.015347,0.108180,-0.006431,-0.169105,0.050339,-0.351655,-0.038061,-0.088107,...,-0.559591,-0.140938,0.134098,0.095605,0.095605,-0.101889,0.034112,-0.161085,-0.152712,-0.019863
20230213,0.055892,-0.086400,-0.013403,0.117962,0.004982,-0.073960,0.036941,-0.357210,-0.039494,-0.102167,...,-0.555345,-0.136658,0.144849,0.070707,0.070707,-0.098890,0.033908,-0.146140,-0.164127,0.013341
20230214,0.061372,-0.091677,-0.009527,0.141575,0.002141,-0.065197,0.038866,-0.355355,-0.039971,-0.096284,...,-0.560975,-0.146876,0.162894,0.080155,0.080155,-0.095652,0.054417,-0.146140,-0.154964,0.022734


In [4]:
#factors
# Fit the PCA model for sample data
model = PCA(n_components=num_components).fit(sample) #???初始设置k=3训练 和 k=n训练后取前3组 有什么差别?

# 得到每个交易日的评分以反映市场(数值大小似乎没有意义,主要是为了之后与symbol做线性回归算残差)
factors = np.dot(sample, model.components_.T)
factors

array([[-1.59370269,  1.91380918,  0.05515138],
       [-1.57079008,  1.86921485,  0.07710862],
       [-1.53716741,  1.8588181 ,  0.08039928],
       ...,
       [ 0.97769503,  0.23660715, -0.35014406],
       [ 0.99463531,  0.25367633, -0.34534145],
       [ 1.05593868,  0.30406217, -0.34804312]])

In [11]:
#resids
# Add 1's to fit the linear regression (intercept) 线性回归的必要处理
factors = sm.add_constant(factors)

# Train Ordinary Least Squares linear model for each stock sample[ticker]为y factors为x 做线性回归
OLSmodels = {ticker: sm.OLS(sample[ticker], factors).fit() for ticker in sample.columns}

# Get the residuals from the linear regression after PCA for each stock
resids = pd.DataFrame({ticker: model.resid for ticker, model in OLSmodels.items()}) #???残差均值很小,计算机标准化过程中似乎会出现数值错误?
resids

,601668.SH,600905.SH,601857.SH,601899.SH,601669.SH,600031.SH,600048.SH,601919.SH,600030.SH,601166.SH,...,603501.SH,603986.SH,601628.SH,688599.SH_x,688599.SH_y,601995.SH,600309.SH,603288.SH,603260.SH,600809.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
20210610,0.000083,-0.393008,0.000361,0.042842,0.108540,0.019171,0.128631,-0.071901,0.066311,0.066824,...,-0.049274,-0.196874,0.028260,-0.116829,-0.116829,0.014033,-0.025612,0.051392,0.067517,0.169995
20210611,0.008768,-0.301499,0.052958,0.035844,0.096169,-0.007376,0.121008,-0.031229,0.040099,0.042544,...,-0.034118,-0.200580,-0.003440,-0.091636,-0.091636,0.034138,-0.012384,0.047427,0.076456,0.143363
20210615,-0.005297,-0.205648,0.059929,-0.012442,0.071787,-0.006675,0.086242,-0.014895,0.028863,0.013449,...,0.018323,-0.217375,-0.003150,-0.076975,-0.076975,0.036744,-0.009368,0.029116,0.066498,0.180206
20210616,-0.013873,-0.123001,0.073439,-0.060581,0.090154,-0.016101,0.084320,-0.051542,0.053669,0.009473,...,0.005393,-0.210565,0.012880,-0.100440,-0.100440,0.032465,-0.034865,0.069552,0.094195,0.174406
20210617,-0.021210,-0.035526,0.056810,-0.071313,0.069813,-0.022616,0.087320,0.030354,0.039452,0.001329,...,0.022475,-0.131834,0.014517,-0.093330,-0.093330,0.044322,-0.057887,0.043088,0.131761,0.209660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230210,0.043057,-0.011762,0.004920,0.169503,0.007612,-0.013350,0.021527,-0.120449,0.057353,0.015510,...,0.023919,0.096379,0.045468,0.042520,0.042520,-0.007091,0.055997,-0.023082,-0.058744,-0.001191
20230213,0.053944,-0.012320,0.009899,0.178866,0.036506,0.066342,0.020369,-0.138281,0.049932,0.003493,...,0.014610,0.094473,0.048529,0.036577,0.036577,-0.016104,0.045477,-0.019663,-0.068265,0.021619
20230214,0.062482,-0.019343,0.014682,0.203001,0.031185,0.070382,0.026243,-0.134933,0.046162,0.012554,...,0.007939,0.083436,0.063631,0.038307,0.038307,-0.016457,0.062350,-0.021877,-0.073471,0.030354


In [20]:
#检测white noise
from statsmodels.stats.diagnostic import acorr_ljungbox as lb_test

white_noise = []
for ticker in resids.columns:
    df = lb_test(resids[ticker], lags=20)
    if not df[df['lb_pvalue'] > 0.05].empty:
        white_noise.append(ticker)

resids.drop(columns=white_noise, inplace=True)

In [9]:
#zscores.sort_values()
# 对残差标准化后取最近一个交易日的zscores  理解的是 zscores负得多,说明对应股票价值被低估(close < 模型认为的股票价值),应该加大下一天对其的投资占比
# Get the Z scores by standarize the given pandas dataframe X  
zscores = ((resids - resids.mean()) / resids.std()).iloc[-1] # residuals of the most recent day
zscores.sort_values()

601919.SH     -1.502757
600438.SH     -1.467898
603799.SH     -1.337161
601012.SH     -1.314786
600196.SH     -1.099676
601225.SH     -0.815931
600893.SH     -0.811406
601888.SH     -0.800155
603260.SH     -0.668686
600104.SH     -0.607750
601088.SH     -0.508056
601688.SH     -0.385585
603259.SH     -0.385033
600900.SH     -0.347997
603288.SH     -0.345698
600905.SH     -0.336559
601995.SH     -0.302993
600048.SH     -0.192216
601066.SH     -0.153087
600887.SH     -0.140214
600346.SH     -0.075065
601633.SH     -0.028764
601857.SH      0.197960
600406.SH      0.220054
601166.SH      0.249270
600809.SH      0.370525
601669.SH      0.382436
603501.SH      0.420090
601628.SH      0.519273
688599.SH_y    0.538583
688599.SH_x    0.538583
600745.SH      0.543521
600111.SH      0.635704
600585.SH      0.746426
600276.SH      0.790561
600030.SH      0.826415
600036.SH      0.922464
600690.SH      0.972634
601668.SH      1.047490
600031.SH      1.127917
603986.SH      1.146771
600309.SH      1

In [10]:
#weights.sort_values()
# Get the stocks far from mean (for mean reversion) 
selected = zscores[zscores < -1.4]

# Return the weights for each selected stock
weights = selected * (1 / selected.sum())

weights.sort_values()

600438.SH    0.494133
601919.SH    0.505867
Name: 20230216, dtype: float64